In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import nltk

nltk.download("inaugural")

[nltk_data] Downloading package inaugural to /home/vscode/nltk_data...
[nltk_data]   Package inaugural is already up-to-date!


True

In [3]:
from pathlib import Path
from typing import Final, TypeAlias

import numpy as np
import pandas as pd
from nltk.corpus import inaugural

from authorship_tool.types_ import Para2dStr
from authorship_tool.util import dim_reshaper
from authorship_tool.util.path_util import InauguralPaths

Project root: /workspaces/shap-authorship-analysis-demo
Path: dump/text_data = /workspaces/shap-authorship-analysis-demo/dump/text_data
Path: data/john_blake_2023/wordLists/adjectivesPastParticiple = /workspaces/shap-authorship-analysis-demo/data/john_blake_2023/wordLists/adjectivesPastParticiple
Path: data/liyanage_vijini_2022 = /workspaces/shap-authorship-analysis-demo/data/liyanage_vijini_2022
Path: data/uoa-thesis-2014-2017 = /workspaces/shap-authorship-analysis-demo/data/uoa-thesis-2014-2017
Path: dump/processed_text = /workspaces/shap-authorship-analysis-demo/dump/processed_text
Path: dump/dataset = /workspaces/shap-authorship-analysis-demo/dump/dataset
Path: dump/lgbm/model = /workspaces/shap-authorship-analysis-demo/dump/lgbm/model
Path: dump/shap/figure = /workspaces/shap-authorship-analysis-demo/dump/shap/figure


In [4]:
output_path: Final[Path] = InauguralPaths.processed_text_dir

In [5]:
np.seterr(divide="call")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [6]:
PRESIDENT_A = "Obama"
PRESIDENT_B = "Bush"

In [7]:
for idx, file_id in enumerate(inaugural.fileids()):
    print(f"#{idx+1}\t{file_id}")

#1	1789-Washington.txt
#2	1793-Washington.txt
#3	1797-Adams.txt
#4	1801-Jefferson.txt
#5	1805-Jefferson.txt
#6	1809-Madison.txt
#7	1813-Madison.txt
#8	1817-Monroe.txt
#9	1821-Monroe.txt
#10	1825-Adams.txt
#11	1829-Jackson.txt
#12	1833-Jackson.txt
#13	1837-VanBuren.txt
#14	1841-Harrison.txt
#15	1845-Polk.txt
#16	1849-Taylor.txt
#17	1853-Pierce.txt
#18	1857-Buchanan.txt
#19	1861-Lincoln.txt
#20	1865-Lincoln.txt
#21	1869-Grant.txt
#22	1873-Grant.txt
#23	1877-Hayes.txt
#24	1881-Garfield.txt
#25	1885-Cleveland.txt
#26	1889-Harrison.txt
#27	1893-Cleveland.txt
#28	1897-McKinley.txt
#29	1901-McKinley.txt
#30	1905-Roosevelt.txt
#31	1909-Taft.txt
#32	1913-Wilson.txt
#33	1917-Wilson.txt
#34	1921-Harding.txt
#35	1925-Coolidge.txt
#36	1929-Hoover.txt
#37	1933-Roosevelt.txt
#38	1937-Roosevelt.txt
#39	1941-Roosevelt.txt
#40	1945-Roosevelt.txt
#41	1949-Truman.txt
#42	1953-Eisenhower.txt
#43	1957-Eisenhower.txt
#44	1961-Kennedy.txt
#45	1965-Johnson.txt
#46	1969-Nixon.txt
#47	1973-Nixon.txt
#48	1977-Car

In [8]:
President: TypeAlias = str
NumOfParas: TypeAlias = int

presidents: set[President] = {file_id[5:-4] for file_id in inaugural.fileids()}

president_data_dict: dict[President, NumOfParas] = {}

for index, president in enumerate(presidents):
    speeches: list[list[Para2dStr]] = [
        inaugural.paras(fileids=file_id)
        for file_id in inaugural.fileids()
        if president in file_id
    ]

    para_num: NumOfParas = len([para for paras in speeches for para in paras])
    president_data_dict[president] = para_num

sorted_para_size_by_president: dict[President, NumOfParas] = dict(
    sorted(president_data_dict.items(), key=lambda item: item[1], reverse=True)
)

for idx, item in enumerate(sorted_para_size_by_president.items()):
    print(f"{idx + 1}:\t{item[0]} - {item[1]} paragraphs")

1:	Nixon - 128 paragraphs
2:	Roosevelt - 121 paragraphs
3:	Eisenhower - 92 paragraphs
4:	Reagan - 81 paragraphs
5:	Bush - 79 paragraphs
6:	Truman - 71 paragraphs
7:	Obama - 66 paragraphs
8:	Harrison - 60 paragraphs
9:	Clinton - 50 paragraphs
10:	Monroe - 48 paragraphs
11:	Cleveland - 44 paragraphs
12:	Taft - 43 paragraphs
13:	Lincoln - 43 paragraphs
14:	Biden - 41 paragraphs
15:	Garfield - 39 paragraphs
16:	Harding - 34 paragraphs
17:	Hoover - 33 paragraphs
18:	Grant - 32 paragraphs
19:	Trump - 32 paragraphs
20:	McKinley - 30 paragraphs
21:	Johnson - 30 paragraphs
22:	Polk - 30 paragraphs
23:	Wilson - 28 paragraphs
24:	Hayes - 27 paragraphs
25:	Kennedy - 27 paragraphs
26:	Carter - 25 paragraphs
27:	Adams - 24 paragraphs
28:	Coolidge - 24 paragraphs
29:	Jefferson - 22 paragraphs
30:	Jackson - 21 paragraphs
31:	Buchanan - 20 paragraphs
32:	VanBuren - 19 paragraphs
33:	Madison - 18 paragraphs
34:	Pierce - 14 paragraphs
35:	Taylor - 10 paragraphs
36:	Washington - 9 paragraphs


In [9]:
speeches_a: list[list[Para2dStr]] = [
    inaugural.paras(file_id)
    for file_id in inaugural.fileids()
    if PRESIDENT_A in file_id
]

paras_a: list[Para2dStr] = [para for paras in speeches_a for para in paras]

for para in paras_a[:20]:
    print(dim_reshaper.two_dim_to_str(para))

print(f"...\n\nSpeaker: President {PRESIDENT_A}, {len(paras_a)} paragraphs\n")

My fellow citizens :
I stand here today humbled by the task before us , grateful for the trust you have bestowed , mindful of the sacrifices borne by our ancestors . I thank President Bush for his service to our nation , as well as the generosity and cooperation he has shown throughout this transition .
Forty - four Americans have now taken the presidential oath . The words have been spoken during rising tides of prosperity and the still waters of peace . Yet , every so often the oath is taken amidst gathering clouds and raging storms . At these moments , America has carried on not simply because of the skill or vision of those in high office , but because We the People have remained faithful to the ideals of our forbearers , and true to our founding documents .
So it has been . So it must be with this generation of Americans .
That we are in the midst of crisis is now well understood . Our nation is at war , against a far - reaching network of violence and hatred . Our economy is badl

In [10]:
speeches_b: list[list[Para2dStr]] = [
    inaugural.paras(file_id)
    for file_id in inaugural.fileids()
    if PRESIDENT_B in file_id
]

paras_b: list[Para2dStr] = [para for paras in speeches_b for para in paras]

for para in paras_b[:20]:
    print(dim_reshaper.two_dim_to_str(para))

print(f"...\n\nSpeaker: President {PRESIDENT_B}, {len(paras_b)} paragraphs\n")

Mr . Chief Justice , Mr . President , Vice President Quayle , Senator Mitchell , Speaker Wright , Senator Dole , Congressman Michael , and fellow citizens , neighbors , and friends :
There is a man here who has earned a lasting place in our hearts and in our history . President Reagan , on behalf of our Nation , I thank you for the wonderful things that you have done for America .
I have just repeated word for word the oath taken by George Washington 200 years ago , and the Bible on which I placed my hand is the Bible on which he placed his . It is right that the memory of Washington be with us today , not only because this is our Bicentennial Inauguration , but because Washington remains the Father of our Country . And he would , I think , be gladdened by this day ; for today is the concrete expression of a stunning fact : our continuity these 200 years since our government began .
We meet on democracy ' s front porch , a good place to talk as neighbors and as friends . For this is a 

In [11]:
print(f"total: {len(paras_a + paras_b)} samples (paragraphs)")

total: 145 samples (paragraphs)


In [12]:
from authorship_tool.util.serializer import author_collection_to_csv


author_collection_to_csv(speeches_a, PRESIDENT_A, output_path)
author_collection_to_csv(speeches_b, PRESIDENT_B, output_path)

+---------------+------------------+-------------------+------------------+----------+----------+-----------+-----------+----------+----------+----------+----------+--------------+------------+-----------+-----------+-----------+-----------+-----------+------------+-----------+-------------+-----------+-----------+-----------+------------+------------+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+---------